## Manim - Clase 3

In [1]:
from manim import *

### Sincronización de animaciones

Algo que hasta ahora nos vino faltando es algún elemento que nos permita sincronizar animaciones, de modo de tener en pantalla varias cosas ocurriendo al mismo tiempo, coordinadas. 

Uno de los principales recursos para lograr esto proviene de la combinación de un objeto de clase `ValueTracker` con la función `always_redraw`. 

Un `ValueTracker` es esencialmente una variable numérica, con la particularidad de que nos permite informar a otros elementos de sus cambios de valor. Veamos un ejemplo:

### Primera Animación

Recomendamos correr el siguiente bloque y mirar la animación resultante antes de leer el código. Ayuda tener en mente el resultado para comprender la lógica subyacente. 

In [2]:
%%manim -qm C3A00

class C3A00(Scene):
    def construct(self):
        ax         = Axes(x_range=[-4, 4],
                          y_range=[-5, 5])
        a          = ValueTracker(1)
        f          = lambda x: a.get_value() * x**2
        func       = always_redraw(lambda: ax.plot(f, color=BLUE))
        
        formula_f  = MathTex("f(x)=ax^2").to_corner(UL)
        etiqueta_a = MathTex("a=")
        etiqueta_a.next_to(formula_f, DOWN).align_to(formula_f, LEFT, LEFT)
        valor_a    = always_redraw(lambda: MathTex(str('%.2f' % (a.get_value()))).next_to(etiqueta_a, RIGHT))

        self.play(Create(ax))
        self.play(Create(func))
        self.play(FadeIn(etiqueta_a), FadeIn(valor_a), FadeIn(formula_f))
        
        self.play(a.animate.set_value(2), runtime=2)
        self.wait()
        self.play(a.animate.set_value(1), runtime=2)
        self.play(a.animate.set_value(0.5), runtime=2)
        self.wait()
        self.play(a.animate.set_value(-1), runtime=2)
        self.wait()
        self.play(a.animate.set_value(-2), runtime=2)
        self.wait()

#### Análisis

El video hace algo de lo que nos proponemos: define una cuadrática de la forma $f(x)=ax^2$ y luego muestra el efecto de cambiar el parámetro $a$. Esto implica hacer dos cosas: por un lado corregir el dibujo y por el otro modificar el texto en el que se muestra el valor de $a$. 

¿Cómo se logra esto? 

Definimos `a = ValueTracker(1)`. Es decir que la variable `a` es un `ValueTracker` cuyo valor inicial es 1. Luego, definimos la función `f` utilizando como coeficiente `a.get_value()`. Esto extrae el valor del `ValueTracker` `a`. Por ahora ese valor es 1 y por lo tanto la función es $x^2$. Pero es importante tener en cuenta que ese 1 no está fijo, sino que proviene de tomar el valor de `a`, que podría cambiar. 

Luego, definimos el gráfico de la función usando `ax.plot`. Pero esta definición la hacemos dentro de `always_redraw`:

    func       = always_redraw(lambda: ax.plot(f, color=BLUE))

`always_redraw` debe recibir una función. Por eso escribimos `lambda:`. Esta función no tiene ningún parámetro, pero *devuelve* el gráfico de la función `f` (que a su vez depende del valor de `a`). `always_redraw` es una función propia de Manim que se corre en cada cuadro de la animación. Es decir que cada vez que Manim genere un cuadro del video va a *redibujar* lo que nosotros le pedimos: en este caso, el gráfico de `f`. Esto hace que el gráfico no sea una figura estática, sólo susceptible de ser movida, rotada o transformada a través de la aplicación explícita de alguna de las funciones de Manim. Por el contrario: el gráfico todo el tiempo es redibujado y puede cambiar si los parámetros que lo definen cambian. Si pasamos por alto la parte del texto y saltamos a las animaciones, podemos observar que fuera de la creación de los distintos elementos (ejes, gráfico, texto), las únicas animaciones son del tipo: 

    self.play(a.animate.set_value(2))

Es decir: no estamos animando explícitamente el gráfico de la función. Lo que estamos animando es el cambio de valor de `a`. `a` pasa de su valor original (1) a valer 2. Como el `set_value()` lo ponemos luego de `animate` este proceso de cambio es animado, es decir que `a` transita por un montón de valores intermedios para pasar de 1 a 2. Cada valor intermedio de `a` generará un nuevo cuadro en el video y ese cuadro, gracias al uso de `always_redraw` implicará la recreación del gráfico de `f`, que a su vez ha cambiado porque depende del (nuevo) valor de `a`. Si se observa el código se pueden ver 5 animaciones de cambios de valor de `a`:  de 1 a 2, de vuelta a 1, a 0.5, a -1 y a -2. En el video pueden verse pequeñas pausas luego de cada movimiento de `a`. 

Si ahora volvemos atrás a mirar el texto, podemos encontrar un par de elementos conocidos: definimos `formula_f` usando `MathTex` y la posicionamos en la esquina superior izquierda; definimos `label_a` cuyo texto (estático) es `a = ` y la posicionamos debajo de `formula_a` y alineada en el borde izquierdo. Por último, definimos `valor_a` que es el texto cambiante en el video. Para esto, usamos nuevamente `always_redraw`, sólo que el objeto que queremos crear no es un `plot` sino algo de tipo `MathTex`. El contenido de ese objeto de tipo `MathTex` es:

    str('%.2f' %(a.get_value()))

Esto es simplemente un tecnicismo (útil, pero tecnicismo al fin). No podemos poner `MathTex(a.get_value())`, porque `a.get_value()` devuelve un número (de tipo `float`) y no texto y por lo tanto `MathTex` no es capaz de procesarlo. Una alternativa es escribir `str(a.get_value())` que convierte el valor de `a` en un *string* (cadena de caracteres). El problema es que `a` puede ser un número con muchísimos decimales que **no** queremos ver en pantalla. La sintaxis que usamos simplemente le indica a `str` que debe tomar el valor definido luego de `%` y traducirlo a texto, con punto decimal y dos dígitos detrás de ese punto. La `f` viene de `float` (número flotante). Pueden jugar con esta configuración. Por ejemplo:  probar `str(a.get_value())`, cambiar el número de decimales, etc. 

De esta manera logramos *sincronizar* dos animaciones distintas: la del gráfico y la del texto. Pero en realidad, no sincronizamos dos animaciones: sólo hicimos **una** animación, que es la del cambio de valor de `a`. Lo que ocurre es que tanto el texto como el gráfico dependen de `a` y se actualizan gracias a `always_redraw`. 

### Segunda Animación

Siguiendo la misma lógica podemos animar, por ejemplo, un punto moviéndose a lo largo del gráfico de una función. 

In [ ]:
%%manim -qm C3A01

class C3A01(Scene):
    def construct(self):
        ax = Axes(x_range=[-3, 3],
                  y_range=[-0.5, 3],
                  x_length=8,
                  y_length=5)
        labels = ax.get_axis_labels(x_label="x", y_label="f(x)")
        labels[0].scale(0.5)
        labels[1].scale(0.5)
        
        def f(x): return 2*np.exp(-x**2)
        grafico = ax.plot(f, color=BLUE)
        
        x0      = -1
        t   = ValueTracker(x0)
        dot = always_redraw(lambda : Dot(ax.c2p(t.get_value(), f(t.get_value())), color=RED))
        
        self.play(Create(ax), Create(labels))
        self.wait()
        self.play(Create(grafico), runtime=2)
        self.wait()
        self.play(FadeIn(dot))
        self.wait()
        self.play(t.animate.set_value(1), runtime=2)
        self.wait()

#### Análisis

Creamos un par de ejes y le agregamos sus correspondientes etiquetas. A estas etiquetas les aplicamos la función `scale` (propia de todo Mobject) que nos permite cambiarle el tamaño *in situ*. En este caso las achicamos a la mitad de su tamaño estandar. 

Lo interesante viene después. Definimos un `ValueTracker`, `t`. Si bien esto no se indica en ningún lado, conceptualmente `t` será la coordenada $x$ del punto. Luego, definimos el punto (la variable `dot`) dentro de `always_redraw`. A diferencia de lo que hicimos en otras oportunidades, aquí aprovechamos que `Dot()` puede recibir como parámetros las coordenadas con la ubicación del punto, que en esta caso provienen de aplicar `ax.c2p()` (o `coords_to_point()`) a $(t,f(t))$. Alternativamente, podríamos haber hecho: 

    Dot(color=RED).move_to(ax.c2p(t.get_value(),f(t.get_value())))

**EJERCICIO 1:** Agregar al video dos puntos: uno que se mueva sobre el eje $x$ y otro que se mueva sobre el eje $y$, siguiendo las coordenadasde punto rojo que recorre la curva. 

**EJERCICIO 2:** Cambiar el punto por alguna figura geométrica. Eliminar del video los ejes y el gráfico y constatar que este recurso puede utilizarse para mover Mobjects por una trayectoria prefijada. 

### Tercera Animación

Existe otro mecanismo que permite obtener un efecto análogo al de `always_redraw`. Se trata de los *updaters*. Un *updater* es una función (que definimos nosotros) que se le agrega a un Mobject y se ejecuta en cada cuadro. Rehacemos la animación anterior con un *updater*:

In [3]:
%%manim -qm C3A02

class C3A02(Scene):
    def construct(self):
        ax = Axes(x_range=[-3, 3],
                  y_range=[-0.5, 3],
                  x_length=8,
                  y_length=5)
        labels = ax.get_axis_labels(x_label="x", y_label="f(x)")
        
        def f(x): return 2*np.exp(-x**2)
        grafico = ax.plot(f, color=BLUE)
        x0  = -1
        dot = Dot(color=RED).move_to(ax.c2p(x0, f(x0)))
        t   = ValueTracker(x0)

        def actualizacion(m):
            m.move_to(ax.c2p(t.get_value(), f(t.get_value())))
            
        dot.add_updater(actualizacion)
        
        self.play(Create(ax), Create(labels))
        self.wait()
        self.play(Create(grafico), runtime=2)
        self.wait()
        self.play(FadeIn(dot))
        self.wait()
        self.play(t.animate.set_value(1), run_time=2)
        self.wait()

#### Análisis

La función `actualizacion` recibe un parámetro `m` y le aplica la función `move_to()` hacia las coordenadas determinadas por el `ValueTracker` `t`.  En principio, la función `actualizacion` es genérica y puede aplicarse a cualquier Mobject (si `m` no es un Mobject el programa dará un error al no encontrar la función `move_to` dentro de `m`). Pero luego hacemos: 

    dot.add_updater(actualizacion)

que le agrega a `dot` la función `actualizacion`, como un *updater*. A partir de aquí, la función `actualizacion` de correrá en cada cuadro sobre el Mobject `dot`.  

También existe una función `remove_updater` que permite quitar el *updater* y de este modo interrumpir la actualización continua del Mobject. 

**Ejercicio 3:** Usando `ValueTracker` y `always_redraw` implementar una animación en la que: aparezcan tres cuadrados, uno junto al otro. Y luego los cuadrados roten, a distinta velocidad: concretamente, cada cuadrado debe rotar al doble de velocidad que su vecino de la izquierda. 

### Cuarta Animación: Tangentes

La clase `TangentLine`  permite trazar tangentes a cualquier Mobject. El constructor recibe como parámetros:
* El Mobject al que le queremos trazar una tangente
* Un valor `alpha` entre 0 y 1 que representa la proporción de Mobject a la cual se traza la tangente (e.g.: `alpha`=0.5 traza la tangente en la mitad del Mobject)
* Opcionalmente, `color` y `length` pueden usarse para fijar el color de la tangente y su longitud. 

Además, aplicando la combinación `ValueTracker`+`always_redraw` es posible obtener tangentes que se mueven a lo largo del Mobject o junto con él, manteniéndose tangentes. 

Recomendamos mirar el siguiente video antes de detenerse en el código. 


In [ ]:
%%manim -qm C3A03

class C3A03(Scene):
    def construct(self):
        circulo  = Circle(2,color=BLUE).shift(1.5*DOWN)
        p        = ValueTracker(0)
        tang1    = TangentLine(circulo,0.2,color=YELLOW,length=4)
        tang2    = always_redraw(lambda: TangentLine(circulo,p.get_value(),color=PURPLE,length=4))
        
        texto1   = Text("Tangente en un punto, estática").to_edge(UP,buff=0.5)
        texto2   = Text("Hm...").move_to(texto1.get_center())
        texto3   = Text("Tangente con always_redraw").move_to(texto1.get_center())
        texto4   = Text("Movemos el parámetro").next_to(texto3,DOWN)
        texto5   = Text("Movemos el círculo").move_to(texto4.get_center())
        texto6   = Text("Ahora sí").move_to(texto1.get_center())
        
        self.play(Create(circulo))
        self.play(Create(tang1),Write(texto1))
        self.wait(3)
        self.play(circulo.animate.shift(RIGHT))
        self.play(ReplacementTransform(texto1,texto2))
        self.wait()
        self.play(circulo.animate.shift(LEFT))
        self.play(FadeOut(tang1),FadeOut(texto2))
        self.wait()
        self.play(Create(tang2),Write(texto3))
        self.wait()
        self.play(Write(texto4))
        self.play(p.animate.set_value(0.8))
        self.play(p.animate.set_value(0.3))
        self.play(ReplacementTransform(texto4,texto5))
        self.play(circulo.animate.shift(RIGHT))
        self.play(circulo.animate.shift(UP),p.animate.set_value(0.6))
        self.play(circulo.animate.move_to(ORIGIN),p.animate.set_value(0))
        self.play(FadeOut(texto5))
        self.play(ReplacementTransform(texto3,texto6))
        self.wait()
        self.play(FadeOut(*self.mobjects))

#### Análisis 

Trazamos dos tangentes al círculo. La primera, amarilla, es creada de manera estática con `alpha`=0.2. Al mover el círculo, la recta queda en su lugar y deja de ser tangente al círculo. La segunda, violeta, se crea dentro de un `always_redraw` con un valor de `alpha` que depende del valor del `ValueTracker` `p`. Al cambiar el valor de `p`, la recta se mueve a lo largo del círculo. Al mover el círculo, la recta lo acompaña. Esto es porque en cada cuadro la recta se redibuja como una tangente al círculo en el punto de proporción `p`. 

### Quinta Animación

El gráfico de una función es un Mobject y por lo tanto `TangentLine` puede usarse para trazar tangentes a gráficos de funciones. El único inconveniente es que no es fácil determinar cuál es la proporción de avance a lo largo del gráfico que corresponde a determinado punto. Felizmente, existe la función (propia de cualquier Mobject) `proportion_from_point` que recibe un punto del Mobject y devuelve el valor de `alpha` que le corresponde. Usamos este recurso para trazar la tangente a un gráfico en un punto dado: 

In [4]:
%%manim -qm C3A04

class C3A04(Scene):
    def construct(self):
        ax = Axes(x_range=[-3, 3],
                  y_range=[-0.5, 3],
                  x_length=8,
                  y_length=5)
        labels = ax.get_axis_labels(x_label="x", y_label="f(x)")
        labels[0].scale(0.5)
        labels[1].scale(0.5)
        def f(x): return 2*np.exp(-x**2)
        grafico = ax.plot(f, x_range=[-2, 2], color=BLUE)
        x0 = -1
        dot = Dot(ax.c2p(x0, f(x0)), color=RED)
        tan = TangentLine(grafico, grafico.proportion_from_point(
                            ax.c2p(x0, f(x0))), color=YELLOW, length=6)
        self.play(Create(ax), Create(labels))
        self.wait()
        self.play(Create(grafico), runtime=2)
        self.wait()
        self.play(FadeIn(dot))
        self.play(Create(tan))
        self.wait()

#### Análisis

Como anticipamos: ubicamos un punto en el gráfico y usamos la función `proportion_from_point` para pasarle el valor de `alpha` correcto a la tangente. 

### Sexta Animación

Ahora podemos tomar la construcción anterior y animar el movimiento de la tangente (junto con el punto), a lo largo de la curva. Para ello definiremos un `ValueTracker` que representará el valor de `alpha`:

In [5]:
%%manim -qm C3A05

class C3A05(Scene):
    def construct(self):
        ax = Axes(x_range=[-3, 3],
                  y_range=[-0.5, 3],
                  x_length=8,
                  y_length=5)
        labels = ax.get_axis_labels(x_label="x", y_label="f(x)")
        labels[0].scale(0.5)
        labels[1].scale(0.5)
        
        def f(x): return 2*np.exp(-x**2)
        grafico = ax.plot(f, color=BLUE)
        #x0  = -1
        dot = Dot(color=RED)
        
        a   = ValueTracker(0.1)

        def update_dot(punto):
            X = grafico.point_from_proportion(a.get_value())
            punto.move_to(X)
            
        dot.add_updater(update_dot)

        def draw_tan():
            return TangentLine(grafico, a.get_value(), color=YELLOW, length=6)
        tan = always_redraw(draw_tan)

        self.play(Create(ax), Create(labels))
        self.wait()
        self.play(Create(grafico), runtime=2)
        self.wait()
        self.play(FadeIn(dot))
        self.play(Create(tan))
        self.wait()
        self.play(a.animate.set_value(0.9), rate_func=linear, run_time=3)
        self.wait()


#### Análisis

Dado que nuestro `ValueTracker` `a` representa la proporción del gráfico en la que se encuentran el punto y la tangente, la definición de la tangente es *fácil*: simplemente tomamos el `a.get_value()` como `alpha`. En este caso optamos por definir la función que le damos a `always_redraw` de manera anticipada, con un `def`, pero podríamos haberlo hecho como antes, directamente dentro de `always_redraw`, con un `lambda:`. Esta versión es un poco más larga en cantidad de líneas, pero permite tener líneas más cortas y claras. 

Para el punto, optamos por utilizar un *updater*. Es importante tener en cuenta que el valor de `a` no es del todo conveniente para fijar un punto en el plano y por lo tanto necesitamos la función `point_from_proportion` (la inversa de `proportion_from_point`).

Está claro que el parámetro `alpha` no es el más práctico cuando trabajamos con el gráfico de una función. Parece más natural controlar, por ejemplo, la coordenada $x$ del punto de interés. En principio, esto podría hacerse usando la función `proportion_from_point` en combinación con `ax.c2p()` para recuperar `alpha`. Sin embargo, no siempre es posible realizar animaciones con este procedimiento. El problema es  que para ciertos valoresde $x$, la conversión de unidades `ax.c2p()` introduce pequeños errores de redondeo que hacen que `proportion_from_point` no reconozca el punto resultante como un integrante de la curva y de un error. 

### Séptima Animación

Está claro que el parámetro `alpha` no es el más práctico cuando trabajamos e